In [2]:
import zarr
import s3fs
import pandas as pd
import numpy as np
import xarray as xr

## Read P-drive dataset

In [4]:
local_url = "/mnt/p/wflow_global/hydromt/meteo/era5_daily.zarr"
ds = xr.open_zarr(local_url, consolidated=False)
ds

<xarray.Dataset>
Dimensions:    (time: 26694, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1950-01-02 1950-01-03 ... 2023-02-01
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    msl        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tisr       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmin       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>

In [31]:
for coord in ("latitude", "longitude", "time"):
    values = ds.tmin.coords[coord].values
    print(coord)
    assert len(np.unique(values)) == len(values)

latitude
longitude
time


In [41]:
zgroup = zarr.open(local_url)
times = list(zgroup.time)
npt = np.asarray(times)
u, c = np.unique(npt, return_counts=True)
dup = u[c > 1]
dup

array([], dtype=int64)

In [42]:
import json
from fsspec.implementations.local import LocalFileSystem
lfs = LocalFileSystem()
jsonmeta = lfs.cat(f"{local_url}/.zmetadata")
zarr_meta = json.loads(jsonmeta.decode())
zarr_meta["metadata"]["time/.zarray"]

{'chunks': [365],
 'compressor': {'blocksize': 0,
  'clevel': 5,
  'cname': 'lz4',
  'id': 'blosc',
  'shuffle': 1},
 'dtype': '<i8',
 'fill_value': None,
 'filters': None,
 'order': 'C',
 'shape': [26694],
 'zarr_format': 2}

In [43]:
zarr_meta["metadata"]["time/.zattrs"]

{'_ARRAY_DIMENSIONS': ['time'],
 'calendar': 'proleptic_gregorian',
 'units': 'days since 1950-01-02'}

## Read Xarray with zarr

In [3]:
# running this twice fails
url = "s3://hydromt-data/meteo/era5_daily.zarr"
fs = s3fs.S3FileSystem()
fs.retries = 10
s3_store = zarr.storage.FSStore(url, mode="w", fs=fs, check=False, create=False)

In [4]:
ds = xr.open_zarr(s3_store, consolidated=True, chunks="auto") # , use_cftime=True,decode_cf=False)
ds

<xarray.Dataset>
Dimensions:    (time: 26694, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1950-01-02 1950-01-03 ... 2023-02-01
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    msl        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tisr       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmin       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>

In [5]:
import json
jsonmeta = fs.cat("s3://hydromt-data/meteo/era5_daily.zarr/.zmetadata")
zarr_meta = json.loads(jsonmeta.decode())
zarr_meta["metadata"]["time/.zarray"]

{'chunks': [365],
 'compressor': {'blocksize': 0,
  'clevel': 5,
  'cname': 'lz4',
  'id': 'blosc',
  'shuffle': 1},
 'dtype': '<i8',
 'fill_value': None,
 'filters': None,
 'order': 'C',
 'shape': [26694],
 'zarr_format': 2}

In [6]:
zarr_meta["metadata"]["time/.zattrs"]

{'_ARRAY_DIMENSIONS': ['time'],
 'calendar': 'proleptic_gregorian',
 'units': 'days since 1950-01-02'}

In [7]:
zgroup = zarr.open(s3_store)
times = list(zgroup.time)
npt = np.asarray(times)
u, c = np.unique(npt, return_counts=True)
dup = u[c > 1]
dup

array([], dtype=int64)

In [8]:
ds.chunks

Frozen({'time': (365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 365, 49), 'latitude': (103, 103, 103, 103, 103, 103, 103), 'longitude': (480, 480, 480)})

In [9]:
ds.tmin.data

dask.array<open_dataset-tmin, shape=(26694, 721, 1440), dtype=float32, chunksize=(365, 103, 480), chunktype=numpy.ndarray>

In [10]:
for coord in ("latitude", "longitude", "time"):
    values = ds.tmin.coords[coord].values
    print(coord)
    assert len(np.unique(values)) == len(values)

latitude
longitude
time


In [16]:
threebyone = ds[["tmin", "tmax"]].sel(
    time=pd.date_range("2020-01-01", "2023-01-01"),
    latitude=np.arange(0, 0.25 * 103, 0.25),
    longitude=np.arange(0, 0.25 * 480, 0.25)
)
threebyone = threebyone.chunk(latitude=103, longitude=480, time=365)
threebyone

<xarray.Dataset>
Dimensions:    (time: 1097, latitude: 103, longitude: 480)
Coordinates:
  * latitude   (latitude) float32 0.0 0.25 0.5 0.75 ... 24.75 25.0 25.25 25.5
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 119.0 119.2 119.5 119.8
  * time       (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2023-01-01
Data variables:
    tmin       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>

In [18]:
# Need mapper for xarray write
write_url = "s3://hydromt-data/threebyone.zarr"
s3map = fs.get_mapper(write_url, check=False, create=False)
store = threebyone.to_zarr(store=s3map)

In [19]:
zarr.consolidate_metadata(write_url)

<zarr.hierarchy.Group '/'>

In [20]:
ds_opened = xr.open_zarr(write_url)
ds_opened

<xarray.Dataset>
Dimensions:    (latitude: 103, longitude: 480, time: 1097)
Coordinates:
  * latitude   (latitude) float32 0.0 0.25 0.5 0.75 ... 24.75 25.0 25.25 25.5
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 119.0 119.2 119.5 119.8
  * time       (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2023-01-01
Data variables:
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>
    tmin       (time, latitude, longitude) float32 dask.array<chunksize=(365, 103, 480), meta=np.ndarray>

In [12]:
ds_opened.coords["time"]

<xarray.DataArray 'time' (time: 1094)>
array(['1950-01-02T00:00:00.000000000', '1950-01-03T00:00:00.000000000',
       '1950-01-04T00:00:00.000000000', ..., '1950-01-02T00:00:00.000000000',
       '1950-01-02T00:00:00.000000000', '1950-01-02T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1950-01-02 1950-01-03 ... 1950-01-02

In [3]:
# Removing cache does not work
# ds.close()
# fs.clear_instance_cache()
# del ds, fs

## Open source zarr GCS

In [2]:
ds_open = xr.open_zarr('gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2', chunks="auto", consolidated=True) # , decode_cf=False)
ds_open

<xarray.Dataset>
Dimensions:              (time: 1084032, values: 542080)
Coordinates:
    depthBelowLandLayer  float64 ...
    entireAtmosphere     float64 ...
    latitude             (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude            (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    number               int64 ...
    step                 timedelta64[ns] ...
    surface              float64 ...
  * time                 (time) datetime64[ns] 1900-01-01 ... 2023-08-31T23:0...
    valid_time           (time) datetime64[ns] dask.array<chunksize=(1084032,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables: (12/38)
    cape                 (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    d2m                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    hcc                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    istl1                (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    istl2                (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    istl3                (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    ...                   ...
    tsn                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    u10                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    u100                 (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    v10                  (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    v100                 (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
    z                    (time, values) float32 dask.array<chunksize=(1, 542080), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              1
    GRIB_subCentre:            0
    history:                   2023-08-27T15:04 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  89c3caaaaf19eb0f6e9749dbde6c93972d64419137e2c8...
    pangeo-forge:recipe_hash:  09c22f5fffc2fbe3742fbb38c8c2c761e1b41b50570f09...
    pangeo-forge:version:      0.9.5.dev2+gd43015b

In [6]:
ds_open.coords["latitude"][9].values

array(89.78487691)

## Write with zarr

In [2]:
url = "hydromt-data/notebooks"
fs = s3fs.S3FileSystem()
s3_store = zarr.storage.FSStore(url, normalize_keys=True, key_separator="/", mode="w", dimension_separator="/", fs=fs, check=False, create=False)

In [3]:
root = zarr.group(store=s3_store, overwrite=True)

In [4]:
z = root.zeros(name="zeros", shape=(100, 100), chunks=(2, 2), dtype='i4')
z.cdata_shape

(50, 50)

In [5]:
# Currently does not contain any metadata
dict(z.attrs)

{}

In [6]:
# zarr needs specific metadata: https://docs.xarray.dev/en/latest/internals/zarr-encoding-spec.html
z.attrs["_ARRAY_DIMENSIONS"] = ["x", "y"]
z.attrs["coordinates"] = "xc yc"
z.attrs["long_name"] = 'zeroes for testing',
# 'time_rep': 'instantaneous',
z.attrs["type_preferred"] = 'uint32',
# 'units': 'C'}

# Then create the dimension arrays
x = root.array("xc", np.arange(0, 100), chunks=(2,), dtype='i4')
x.attrs["_ARRAY_DIMENSIONS"] = ["x"]
x.attrs["long_name"] = "x"
y = root.array("yc", np.arange(0, 100), chunks=(2,), dtype='i4')
y.attrs["_ARRAY_DIMENSIONS"] = ["y"]
y.attrs["long_name"] = "y"

In [7]:
# change a single_value
z[50,50] = 42

In [8]:
# close the storage
s3_store.close()

In [18]:
# Because we use cloud storage, consolidate metadata
zarr.consolidate_metadata("s3://hydromt-data/notebooks")  # cannot use s3_store as argument, as the metadata are then placed under 's3://hydromt-data/notebooks//'

<zarr.hierarchy.Group '/'>

In [19]:
# read again
root = zarr.group(store=s3_store)
z = root['zeros']

In [20]:
z

<zarr.core.Array '/zeros' (100, 100) int32>

In [21]:
z[50,50]

42

In [22]:
z.chunks

(2, 2)

In [23]:
s3_store.close()

### read this using xr

In [28]:
ds_self = xr.open_zarr("s3://hydromt-data/notebooks")
ds_self

<xarray.Dataset>
Dimensions:  (x: 100, y: 100)
Coordinates:
    xc       (x) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    yc       (y) float64 dask.array<chunksize=(2,), meta=np.ndarray>
Dimensions without coordinates: x, y
Data variables:
    zeros    (x, y) float64 dask.array<chunksize=(2, 2), meta=np.ndarray>